In [8]:
import pandas as pd

/var/folders/x5/spm_6r5944dfgx3qj8my0m440000gn/T/ipykernel_11697/4080736814.py:1: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [9]:
from pathlib import Path

def load_data(path: str) -> pd.DataFrame:
    data = pd.read_csv(Path(path))
    return data

In [10]:
data = load_data("../Data/test.csv")

In [27]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3263 entries, 0 to 3262
Data columns (total 4 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   id        3263 non-null   int64 
 1   keyword   3237 non-null   object
 2   location  2158 non-null   object
 3   text      3263 non-null   object
dtypes: int64(1), object(3)
memory usage: 102.1+ KB


In [3]:
from huggingface_hub import notebook_login
notebook_login()

In [5]:
from sentence_transformers import SentenceTransformer

model = SentenceTransformer("intfloat/e5-large-v2")

model.safetensors:   0%|          | 0.00/1.34G [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/314 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/711k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/201 [00:00<?, ?B/s]

In [11]:
embeddings = model.encode(data['text'], normalize_embeddings=True)

In [13]:
embeddings

array([[ 0.03187924, -0.05397046,  0.01730515, ...,  0.01002994,
        -0.0161275 ,  0.00631168],
       [ 0.0171534 , -0.05043046,  0.03901209, ..., -0.02807589,
         0.01735447,  0.02182877],
       [ 0.02056186, -0.06100253,  0.01692475, ..., -0.04115108,
         0.03407306,  0.02501726],
       ...,
       [ 0.01640777, -0.04948715,  0.03447014, ..., -0.02517169,
         0.00732588,  0.03530829],
       [ 0.00367261, -0.06605804,  0.04944976, ..., -0.034654  ,
         0.01319482,  0.01746392],
       [ 0.01460129, -0.02757392,  0.03899997, ..., -0.03441751,
         0.02077394,  0.0131358 ]], dtype=float32)